In [1]:
%run functions.py
#PO
po = loadPO('Data/GF12 All Purchase Orders By Job 1.27.24.xlsx')
machined, purchased = loadBOM('Data/GF12 BOM Master Tab Copies 1.27.24.xlsx')

In [15]:
lookupPartNumber(po, '001283')

,Type,Date,Num,Source Name,Part Number,Item,Item Description,Cost Price,Qty,Pack Qty,Unit Price,Unit Qty,Override 1,Override 2
63,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001283,001283 (BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) an...,BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) and R91134...,1775.07,8.0,1.0,1775.07,8.0,NaN,NaN
64,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001283,001283 (BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) an...,BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) and R91134...,1775.07,2.0,1.0,1775.07,2.0,NaN,NaN
65,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001283,001283 (BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) an...,BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) and R91134...,1775.07,5.0,1.0,1775.07,5.0,NaN,NaN
